In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


class SelfAttention(nn.Module):
    """
    基础Self Attention层实现
    核心逻辑：输入通过Linear映射生成同源Q/K/V，采用批次矩阵乘法优化计算效率
    架构流程：Linear映射→MatMul(Q,K^T)→Scale→Mask（可选）→SoftMax→MatMul(V)
    """
    def __init__(self, dim_q, dim_k, dim_v):
        super(SelfAttention, self).__init__()
        # 输入映射层：将输入分别映射为Q、K、V
        self.linear_q = nn.Linear(dim_q, dim_k, bias=False)  # 输入→Q
        self.linear_k = nn.Linear(dim_q, dim_k, bias=False)  # 输入→K（与Q维度一致）
        self.linear_v = nn.Linear(dim_q, dim_v, bias=False)  # 输入→V
        
        # 缩放因子：避免注意力分数过大导致SoftMax梯度消失
        self._norm_fact = 1 / math.sqrt(dim_k)
        
        # 记录初始化维度，用于后续输入校验
        self.dim_q = dim_q
        self.dim_k = dim_k
        self.dim_v = dim_v

    def forward(self, x, mask=None):
        """
        前向传播
        Args:
            x: 输入张量，形状[batch_size, seq_len, dim_q]（Self Attention同源输入）
            mask: 可选掩码张量，形状[batch_size, seq_len, seq_len]（用于屏蔽无效序列位置）
        
        Returns:
            att: Self Attention输出张量，形状[batch_size, seq_len, dim_v]
            attn_weights: 注意力权重张量，形状[batch_size, seq_len, seq_len]（用于后续分析）
        """
        # 获取输入维度并校验，确保与初始化维度一致
        batch_size, seq_len, dim_q = x.shape
        assert dim_q == self.dim_q, f"输入维度{dim_q}与初始化dim_q{self.dim_q}不一致"
        
        # 生成Q、K、V：输入通过Linear层完成维度映射
        q = self.linear_q(x)  # [batch_size, seq_len, dim_k]
        k = self.linear_k(x)  # [batch_size, seq_len, dim_k]
        v = self.linear_v(x)  # [batch_size, seq_len, dim_v]
        
        # 步骤1：计算注意力分数并缩放（批次内矩阵乘法优化效率）
        attn_scores = torch.bmm(q, k.transpose(1, 2)) * self._norm_fact  # [batch_size, seq_len, seq_len]
        
        # 步骤2：应用掩码（屏蔽无效位置，掩码为0处设为极小值）
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # 步骤3：SoftMax归一化，得到注意力权重
        attn_weights = F.softmax(attn_scores, dim=-1)  # [batch_size, seq_len, seq_len]
        
        # 步骤4：注意力权重与V相乘，生成最终输出
        att = torch.bmm(attn_weights, v)  # [batch_size, seq_len, dim_v]
        
        return att, attn_weights


# 测试Self Attention层功能
if __name__ == "__main__":
    # 1. 配置测试参数（模拟Transformer Encoder输入场景）
    batch_size = 2    # 批次大小
    seq_len = 16      # 序列长度（如文本序列、视觉patch序列数量）
    dim_q = 512       # 输入维度（对应Transformer的d_model）
    dim_k = 64        # Q/K维度（注意力头维度，常规设为d_model/num_heads）
    dim_v = 64        # V维度（与Q/K维度一致，符合Self Attention设计习惯）
    
    # 2. 生成随机输入张量
    x = torch.randn(batch_size, seq_len, dim_q)  # [2, 16, 512]
    
    # 3. 生成掩码（屏蔽序列后8个位置，测试掩码功能）
    mask = torch.ones(batch_size, seq_len, seq_len)  # [2, 16, 16]
    mask[:, :, 8:] = 0  # 屏蔽每个序列的后8个无效位置
    
    # 4. 初始化并执行Self Attention前向传播
    self_attn = SelfAttention(dim_q=dim_q, dim_k=dim_k, dim_v=dim_v)
    att_output, att_weights = self_attn(x, mask=mask)
    
    # 5. 验证输出结果
    print("=== Self Attention层测试结果 ===")
    print(f"输入形状：{x.shape}")
    print(f"输出形状：{att_output.shape}（预期：[{batch_size}, {seq_len}, {dim_v}]）")
    print(f"注意力权重形状：{att_weights.shape}（预期：[{batch_size}, {seq_len}, {seq_len}]）")
    print(f"掩码有效性验证：第一批次第一序列后8位权重均值 = {att_weights[0, 0, 8:].mean():.6f}（接近0为正常）")

=== Self Attention层测试结果 ===
输入形状：torch.Size([2, 16, 512])
输出形状：torch.Size([2, 16, 64])（预期：[2, 16, 64]）
注意力权重形状：torch.Size([2, 16, 16])（预期：[2, 16, 16]）
掩码有效性验证：第一批次第一序列后8位权重均值 = 0.000000（接近0为正常）
